# Managing conversation history with the ChatGPT model
This sample notebook demonstrates a couple of simple patterns you can use for managing the prompts and conversation history with the ChatGPT model.

In [2]:
# if needed, install and/or upgrade to the latest version of the OpenAI Python library
pip install --upgrade #FIXME

  Using cached openai-1.59.8-py3-none-any.whl.metadata (27 kB)
Using cached openai-1.59.8-py3-none-any.whl (455 kB)
  Attempting uninstall: openai
    Found existing installation: openai 1.59.7
    Uninstalling openai-1.59.7:
      Successfully uninstalled openai-1.59.7
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastapi-app 1.0.0 requires tiktoken<0.8.0,>=0.7.0, but you have tiktoken 0.8.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [4]:
# import os module & the OpenAI Python library for calling the OpenAI API
# please make sure you have installed required libraries via pip install -r requirements.txt
import os, dotenv
import openai

dotenv.load_dotenv()

True

In [8]:
# Setting up the deployment name
deployment_name = #FIXME

# The base URL for your Azure OpenAI resource. e.g. "https://<your resource name>.openai.azure.com"
# This is the value of the endpoint for your Azure OpenAI resource
azure_endpoint = os.environ['AZURE_OPENAI_ENDPOINT']

# The API key for your Azure OpenAI resource.
api_key = #FIXME

# Currently OPENAI API have the following versions available: https://learn.microsoft.com/azure/ai-services/openai/reference
api_version = "2024-07-01-preview"

client = #FIXME

## 1.0 Create the system message for ChatGPT

In [11]:
base_system_message = "You are a helpful assistant."

system_message = f"{base_system_message.strip()}"
print(system_message)

You are a helpful assistant.


## 2.0 Define helper functions



In [14]:
import tiktoken

def num_tokens_from_messages(messages, model="gpt-3.5-turbo-0301"):
    encoding = #FIXME
    num_tokens = 0
    for message in messages:
        num_tokens += 4  # every message follows <im_start>{role/name}\n{content}<im_end>\n
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":  # if there's a name, the role is omitted
                num_tokens += -1  # role is always required and always 1 token
    num_tokens += 2  # every reply is primed with <im_start>assistant
    return #FIXME

In [16]:
# Defining a function to send the prompt to the ChatGPT model
# More info : https://learn.microsoft.com/en-us/azure/cognitive-services/openai/how-to/chatgpt?pivots=programming-language-chat-completions
def send_message(messages, model_name, max_response_tokens=500):
    response = client.chat #FIXME
    return #FIXME

# Defining a function to print out the conversation in a readable format
def print_conversation(messages):
    for message in messages:
        print(f"[{message['role'].upper()}]")
        print(message['content'])
        print()

## 3.0 Start the conversation

In [19]:
# This is the first user message that will be sent to the model. Feel free to update this.
user_message = "I want to write a blog post about the impact of AI on the future of work."

In [21]:
# Create the list of messages. role can be either "user" or "assistant" 
messages= #FIXME

In [23]:
token_count = num_tokens_from_messages(messages)
print(f"Token count: {token_count}")

Token count: 37


In [25]:
max_response_tokens = 100

response = send_message(messages, deployment_name, max_response_tokens)
messages.append({"role": "assistant", "content": response})

print_conversation(messages)

[SYSTEM]
You are a helpful assistant.

[USER]
I want to write a blog post about the impact of AI on the future of work.

[ASSISTANT]
That sounds like a fascinating topic! Here’s a structured outline and some key points you might consider including in your blog post about the impact of AI on the future of work:

### Title: The Impact of AI on the Future of Work

#### Introduction
- Brief overview of AI and its rapid development.
- Importance of understanding AI's impact on the workforce.
- Purpose of the blog post: to explore how AI is transforming work and what it means for the future.

#### Section 1: The



## 4.0 Continue the conversation

When working with the ChatGPT model, it's your responsibity to make sure you stay within the token limits of the model. The model can handle a maximum of 4096 tokens, and this includes the number of tokens in the prompt as well as the `max_tokens` you're requesting from the model. If you exceed these limits, the model will return an error.

You should also consider the trade-off between maintaining more of the conversation history and the cost/latency that you'll incur by including those tokens in the prompt. Shorter prompts are cheaper and faster. The amount of the previous conversation you include also makes a difference in how the model responds.

In this notebook, we'll show two strategies for managing the conversation history when working with the ChatGPT model.
- Option 1: Keep the conversation within a given token limit
- Option 2: Keep the conversation within a given number of turns

### Option 1: Keep the conversation within a given token limit

`overall_max_tokens` is the maximum number of tokens that you want to include in the prompt. Th maximum number this can be set to is 4096 but you can also consider reducing this number to reduce the cost and latency of the request.

In [29]:
overall_max_tokens = 1096
prompt_max_tokens = overall_max_tokens - max_response_tokens

You can continue the conversation below by editing the user_message and running the cell as many times as you would like.

In [32]:
user_message = "The target audience for the blog post should be business leaders working in the tech industry."
#user_message = "Let's talk about generative AI and keep the tone informational but also friendly."
#user_message = "Show me a few more examples"
messages.append({"role": "user", "content": user_message})

token_count = num_tokens_from_messages(messages)
print(f"Token count: {token_count}")

# remove first message while over the token limit
while token_count > prompt_max_tokens:
    messages.pop(0)
    token_count = num_tokens_from_messages(messages)

response = #FIXME

#Append assistance response 
messages.append #FIXME
print_conversation(messages)

Token count: 164
[SYSTEM]
You are a helpful assistant.

[USER]
I want to write a blog post about the impact of AI on the future of work.

[ASSISTANT]
That sounds like a fascinating topic! Here’s a structured outline and some key points you might consider including in your blog post about the impact of AI on the future of work:

### Title: The Impact of AI on the Future of Work

#### Introduction
- Brief overview of AI and its rapid development.
- Importance of understanding AI's impact on the workforce.
- Purpose of the blog post: to explore how AI is transforming work and what it means for the future.

#### Section 1: The

[USER]
The target audience for the blog post should be business leaders working in the tech industry.

[ASSISTANT]
Great! Tailoring your blog post for business leaders in the tech industry will allow you to focus on strategic insights and practical implications. Here’s a revised outline with that audience in mind:

### Title: Navigating the Future: The Impact of AI 

### Option 2: Keep the conversation within a given number of turns

In [39]:
max_messages = 10

overall_max_tokens = 4096
prompt_max_tokens = overall_max_tokens - max_response_tokens

You can continue the conversation below by editing the user_message and running the cell as many times as you would like.

In [42]:
user_message = "Make the post about generative AI aimed at business leaders who have some knowledge of the technology."
messages.append({"role": "user", "content": user_message})

while len(messages) > max_messages:
    messages.pop(0)

token_count = num_tokens_from_messages(messages)

while token_count > prompt_max_tokens:
    # remove first message from messages
    messages.pop(0)
    token_count = num_tokens_from_messages(messages)

response = #FIXME

#Append assistant response
messages.append #FIXME
print_conversation(messages)


[SYSTEM]
You are a helpful assistant.

[USER]
I want to write a blog post about the impact of AI on the future of work.

[ASSISTANT]
That sounds like a fascinating topic! Here’s a structured outline and some key points you might consider including in your blog post about the impact of AI on the future of work:

### Title: The Impact of AI on the Future of Work

#### Introduction
- Brief overview of AI and its rapid development.
- Importance of understanding AI's impact on the workforce.
- Purpose of the blog post: to explore how AI is transforming work and what it means for the future.

#### Section 1: The

[USER]
The target audience for the blog post should be business leaders working in the tech industry.

[ASSISTANT]
Great! Tailoring your blog post for business leaders in the tech industry will allow you to focus on strategic insights and practical implications. Here’s a revised outline with that audience in mind:

### Title: Navigating the Future: The Impact of AI on Work in the Te

## Ejercicio 1: Modificar el Mensaje del Sistema
Enunciado:
Crea un mensaje del sistema que defina a ChatGPT como un "analista financiero experto". Imprime el mensaje del sistema resultante y utilízalo en un mensaje inicial del usuario que pregunte:
"¿Cuáles son las tendencias actuales en el mercado de acciones tecnológicas?".


In [45]:

# Mensaje del sistema
base_system_message = "You are a financial analyst expert."
system_message = f"{base_system_message.strip()}"
print(system_message)

# Crear lista de mensajes
user_message = "What are the current trends in the tech stock market?"
messages = #FIXME

# Enviar mensaje y recibir respuesta
response = #FIXME

messages.append({"role": "assistant", "content": response})
print_conversation(messages)


You are a financial analyst expert.
[SYSTEM]
You are a financial analyst expert.

[USER]
What are the current trends in the tech stock market?

[ASSISTANT]
As of my last update in October 2023, several trends were shaping the tech stock market. Here are some key trends to consider:

1. **Artificial Intelligence (AI) Boom**: The rise of AI technologies, particularly generative AI and machine learning, has driven significant interest and investment in companies that specialize in AI development and applications. Stocks of companies involved in AI hardware, software, and services have seen substantial growth.

2. **Cloud Computing**: The transition to cloud-based services continues to be a major trend, with companies investing heavily in cloud infrastructure. Major players like Amazon, Microsoft, and Google have seen strong performance due to their cloud offerings.

3. **Cybersecurity**: As cyber threats increase, so does the demand for cybersecurity solutions. Companies in this sector ar

## Ejercicio 2: Contar Tokens en una Conversación Extendida
Enunciado:
Dado un conjunto inicial de mensajes, agrega tres mensajes adicionales:

"Explain the impact of inflation on stock prices."
"What are the top stocks to watch in 2025?"
"Summarize the latest market news in 50 words."
Calcula y muestra el número total de tokens tras añadir cada mensaje.


In [47]:

# Mensajes iniciales
messages = [
    {"role": "system", "content": "You are a helpful assistant specialized in financial topics."},
    {"role": "user", "content": "What are the current trends in the tech stock market?"}
]

# Mensajes adicionales
additional_messages = [
    "Explain the impact of inflation on stock prices.",
    "What are the top stocks to watch in 2025?",
    "Summarize the latest market news in 50 words."
]

# Agregar mensajes y calcular tokens
for msg in additional_messages:
    messages.append({"role": "user", "content": msg})
    token_count = #FIXME
    print(f"Token count after adding '{msg}': {token_count}")


Token count after adding 'Explain the impact of inflation on stock prices.': 48
Token count after adding 'What are the top stocks to watch in 2025?': 65
Token count after adding 'Summarize the latest market news in 50 words.': 82


## Ejercicio 3: Personalizar el Mensaje del Sistema Según el Usuario
Enunciado:
Escribe un programa que permita al usuario definir el rol de ChatGPT mediante una entrada de texto (input). Una vez definido, responde al mensaje:
"What are your key skills as an assistant?"
Usa el mensaje del sistema definido por el usuario.


In [51]:

# Entrada del usuario para definir el mensaje del sistema
user_defined_role = input("Define the role of the assistant: ")
system_message = f"You are {user_defined_role.strip()}."

# Crear lista de mensajes
messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": "What are your key skills as an assistant?"}
]

# Enviar mensaje y recibir respuesta
response = #FIXME

messages.append({"role": "assistant", "content": response})

# Mostrar la conversación
print_conversation(messages)


Define the role of the assistant:  ok


[SYSTEM]
You are ok.

[USER]
What are your key skills as an assistant?

[ASSISTANT]
As an assistant, my key skills include:

1. **Information Retrieval**: I can provide accurate and relevant information on a wide range of topics.
2. **Natural Language Processing**: I understand and generate human-like text, allowing for smooth and coherent conversations.
3. **Problem Solving**: I can help analyze problems and suggest potential solutions or alternatives.
4. **Task Automation**: I can assist with organizing tasks, setting reminders, and providing productivity tips.
5. **Research Assistance**: I can help gather and summarize information from various sources.
6. **Language Translation**: I can translate text between multiple languages.
7. **Writing and Editing**: I can assist with writing, proofreading, and editing text for clarity and coherence.
8. **Learning and Adaptability**: I can adapt to different topics and user preferences over time.

If you have specific needs, feel free to ask!


## Ejercicio 4: Crear un Generador de Respuestas Creativo
Enunciado:
Diseña un programa que permita configurar la creatividad del modelo (temperatura) a través de una entrada del usuario. Una vez configurado, responde a la pregunta:
"Can you suggest an out-of-the-box idea for a startup?"



In [57]:

# Entrada del usuario para definir la creatividad
temperature = float(input("Set the creativity level (0.0 - 1.0): "))
if not (0.0 <= temperature <= 1.0):
    raise ValueError("Creativity level must be between 0.0 and 1.0.")

# Crear lista de mensajes
system_message = "You are a highly creative assistant."
messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": "Can you suggest an out-of-the-box idea for a startup?"}
]

# Enviar mensaje con creatividad ajustada
response = #FIXME

# Mostrar la respuesta
print(response.choices[0].message.content)

Set the creativity level (0.0 - 1.0):  0.8


Sure! How about a startup called "Memory Lane"? 

**Concept:** A personalized, augmented reality (AR) experience that takes users on nostalgic journeys through their own life memories, as well as collective cultural memories. 

**How it Works:**
1. **Memory Mapping:** Users create a digital timeline of important life events through an app, such as birthdays, anniversaries, vacations, and milestones. They can upload photos, videos, and notes associated with each memory.
  
2. **AR


## Ejercicio 5: Cambiar el Tono del Mensaje
Enunciado:
Configura el mensaje del sistema para que ChatGPT responda en un tono humorístico y escribe una respuesta al mensaje:
"Explain quantum computing in a funny way."


In [60]:

# Mensaje del sistema humorístico
system_message = "You are a humorous assistant."
messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": "Explain quantum computing in a funny way."}
]

# Enviar mensaje y recibir respuesta
response = #FIXME

#append assistant response
messages.append #FIXME
print_conversation(messages)


[SYSTEM]
You are a humorous assistant.

[USER]
Explain quantum computing in a funny way.

[ASSISTANT]
Sure! Imagine quantum computing as a party where all the computers are invited, but instead of just hanging out like regular computers, they decide to throw a wild dance-off.

In the classical computing corner, you have traditional computers. They’re like that one friend who insists on following the rules: "I can only be in one place at a time, and I can only dance one way." They can do calculations, but they do them step by step, like a very organized line dance.

Now, over in the quantum corner, things get a bit wacky. These computers are like that friend who shows up in a sequined outfit and starts doing the cha-cha while simultaneously breakdancing and moonwalking. They can be in multiple states at once, thanks to something called superposition. So while the classical computers are busy doing the Macarena, quantum computers are out there doing the entire dance floor at once!

And t

## Ejercicio 6: Resumir Mensajes Previos
Enunciado:
Dado un historial de conversación de 5 mensajes, crea una función que resuma el contenido de los mensajes del usuario en una sola oración.



In [63]:

# Historial de mensajes
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Tell me about AI."},
    {"role": "user", "content": "How does machine learning work?"},
    {"role": "user", "content": "Can you explain neural networks?"},
    {"role": "user", "content": "What are the applications of deep learning?"}
]

# Resumir mensajes del usuario
def summarize_user_messages(messages):
    user_messages = [msg['content'] for msg in messages if msg['role'] == 'user']
    summary_prompt = "Summarize the following messages into one sentence: " + " ".join(user_messages)
    summary = send_message([{"role": "system", "content": "You are a helpful assistant."},
                            {"role": "user", "content": summary_prompt}], deployment_name)
    return summary

# Obtener y mostrar el resumen
summary = summarize_user_messages(messages)
print(f"User messages summary: {summary}")


User messages summary: AI encompasses various techniques, including machine learning, which involves training algorithms on data, neural networks that simulate brain-like structures for processing information, and deep learning applications that range from image and speech recognition to natural language processing.


## Ejercicio 7: Configuración de Temperatura y Tono
Enunciado:
Escribe un programa que cambie la configuración de temperatura a 0.9 para obtener respuestas más creativas y responde al mensaje:
"Suggest a unique title for a blog about AI in education."



In [66]:

# Mensajes iniciales
messages = [
    {"role": "system", "content": "You are a creative assistant."},
    {"role": "user", "content": "Suggest a unique title for a blog about AI in education."}
]

# Ajustar temperatura y enviar mensaje
response = #FIXME

# Mostrar la respuesta
print(response.choices[0].message.content)


"Learning Reimagined: Navigating the Future of Education with AI"
